# Spectral Signal

In [1]:
import numpy as np
from contextlib import contextmanager
from copy import deepcopy

import colour
from colour import (
    CaseInsensitiveMapping,
    CubicSplineInterpolator,
    Extrapolator,
    LinearInterpolator,
    PchipInterpolator,
    SpragueInterpolator,
    warning)

INTERPOLATORS = CaseInsensitiveMapping({
    'Cubic Spline': CubicSplineInterpolator,
    'Linear': LinearInterpolator,
    'Pchip': PchipInterpolator,
    'Sprague': SpragueInterpolator})


@contextmanager
def ndarray_write(a):
    a.setflags(write=True)

    yield a

    a.setflags(write=False)


def fill_nan(a, method='Interpolation'):
    mask = np.isnan(a)

    if method.lower() == 'interpolation':
        a[mask] = np.interp(
            np.flatnonzero(mask),
            np.flatnonzero(~mask),
            a[~mask])
    elif method.lower() == 'zeros':
        a[mask] = 0

    return a


class ContinuousSignal(object):
    def __init__(self,
                 data=None,
                 index=None,
                 interpolation_method='Pchip',
                 extrapolation_method='Constant',
                 extrapolation_left=np.nan,
                 extrapolation_right=np.nan):
        # TODO: Handle unpacking of dicts, arrays, etc...

        self.__values = None
        self.__index = None
        self.__interpolation_method = None
        self.__extrapolation_method = None
        self.__extrapolation_left = None
        self.__extrapolation_right = None

        self.values = data
        self.index = index
        self.interpolation_method = interpolation_method
        self.extrapolation_method = extrapolation_method
        self.extrapolation_left = extrapolation_left
        self.extrapolation_right = extrapolation_right

        self.__create_function()

    @property
    def values(self):
        return self.__values

    @values.setter
    def values(self, value):
        if value is not None:
            if not np.all(np.isfinite(value)):
                warning('"values" variable is not finite, '
                        'unpredictable results may occur!\n{0}'.format(value))
                
            value = np.asarray(value)
            value.setflags(write=False)

        self.__values = value
        self.__create_function()

    @property
    def index(self):
        return self.__index

    @index.setter
    def index(self, value):
        if value is not None:
            if not np.all(np.isfinite(value)):
                warning('"index" variable is not finite, '
                        'unpredictable results may occur!\n{0}'.format(value))

            value = np.asarray(value)
            value.setflags(write=False)

        self.__index = value
        self.__create_function()

    @property
    def interpolation_method(self):
        return self.__interpolation_method

    @interpolation_method.setter
    def interpolation_method(self, value):
        self.__interpolation_method = value
        self.__create_function()

    @property
    def extrapolation_method(self):
        return self.__extrapolation_method

    @extrapolation_method.setter
    def extrapolation_method(self, value):
        self.__extrapolation_method = value
        self.__create_function()

    @property
    def extrapolation_left(self):
        return self.__extrapolation_left

    @extrapolation_left.setter
    def extrapolation_left(self, value):
        self.__extrapolation_left = value
        self.__create_function()

    @property
    def extrapolation_right(self):
        return self.__extrapolation_right

    @extrapolation_right.setter
    def extrapolation_right(self, value):
        self.__extrapolation_right = value
        self.__create_function()

    @property
    def function(self):
        return self.__function

    @function.setter
    def function(self, value):
        raise AttributeError(
            '"{0}" attribute is read only!'.format('function'))

    def __create_function(self):
        if (self.__index is not None and
                    self.__values is not None and
                    self.__interpolation_method is not None and
                    self.__extrapolation_method is not None):
            assert self.__index.size == self.__values.size, (
            '"index" and "values" variables must have same size!')

            self.__function = Extrapolator(
                INTERPOLATORS[self.__interpolation_method](
                    self.__index, self.__values),
                method=self.__extrapolation_method,
                left=self.__extrapolation_left,
                right=self.__extrapolation_right)
        else:
            def __undefined_signal_interpolator_function(*args, **kwargs):
                raise RuntimeError(
                    'Underlying signal interpolator function does not exists, '
                    'please ensure you defined both "index" and "values" variables!')

            self.__function = __undefined_signal_interpolator_function

    def __getitem__(self, x):
        if type(x) is slice:
            return self.__values[x]
        else:
            return self.__function(x)

    def __setitem__(self, x, value):
        if type(x) is slice:
            with ndarray_write(self.__values):
                self.__values[x] = value
        else:
            with ndarray_write(self.__index), ndarray_write(self.__values):
                x = np.atleast_1d(x)
                value = np.atleast_1d(value)

                # Matching index, replacing existing `self.values`.             
                self.__values[np.in1d(self.__index, x)] = value

                # Non matching index, inserting into existing `self.index` and 
                # `self.values`.
                x = x[~np.in1d(x, self.__index)]
                indexes = np.searchsorted(self.__index, x)

                self.__index = np.insert(self.__index, indexes, x)
                self.__values = np.insert(self.__values, indexes, value)

        self.__create_function

    def __iadd__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)

        with ndarray_write(self.__values):
            self.values += x

        return self

    def __add__(self, x):
        copy = self.copy()
        copy += x

        return copy

    def __isub__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)

        with ndarray_write(self.__values):
            self.values -= x

        return self

    def __sub__(self, x):
        copy = self.copy()
        copy -= x

        return copy

    def __imul__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)

        with ndarray_write(self.__values):
            self.values *= x

        return self

    def __mul__(self, x):
        copy = self.copy()
        copy *= x

        return copy

    def __idiv__(self, x):
        if isinstance(x, self.__class__):
            x = self.__function(x.index)

        with ndarray_write(self.__values):
            self.values /= x

        return self

    def __div__(self, x):
        copy = self.copy()
        copy /= x

        return copy

    __itruediv__ = __idiv__
    __truediv__ = __div__

    def copy(self):
        return deepcopy(self)

    def fill_nan(self, method='Interpolation'):
        with ndarray_write(self.__index), ndarray_write(self.__values):
            self.__index = fill_nan(self.__index, method)
            self.__values = fill_nan(self.__values, method)
            self.__create_function()


colour.message_box('Empty Object Creation')
cs1 = ContinuousSignal()
print('cs1[0]')
try:
    print(cs1[0])
except RuntimeError as error:
    print(error)

print('\n')

index = np.arange(0, 1000, 100)
cs1 = ContinuousSignal(index)
print('cs1[0]')
try:
    print(cs1[0])
except RuntimeError as error:
    print(error)

print('\n')

values = np.linspace(1, 10, index.size)
cs1 = ContinuousSignal(values, index)
print('cs1[0]')
print(cs1[0])
    
print('\n')

try:
    cs1 = ContinuousSignal(values, [])
except AssertionError as error:
    print(error)

print('\n')

colour.message_box('Copy Operations')
cs1 = ContinuousSignal(values, index)
print('id(cs1)')
print(id(cs1))
print(cs1.function)

print('\n')

cs2 = cs1.copy()
print('id(cs2)')
print(id(cs2))
print(cs2.function)

print('\n')

colour.message_box('Item Operations')

print('cs1')
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[150.25]')
print(cs1[150.25])

print('\n')

print('cs1[np.linspace(100, 400, 10)]')
print(cs1[np.linspace(100, 400, 10)])

print('\n')

print('cs1[0:3]')
print(cs1[0:3])

print('\n')

print('cs1[10] = np.pi')
cs1[10] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[(200, 300)] = np.pi')
cs1[(200, 300)] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[(0, 850)] = np.pi')
cs1[(0, 850)] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

print('cs1[0:9] = np.pi')
cs1[0:9] = np.pi
print(cs1.values)
print(cs1.index)

print('\n')

colour.message_box('Arithmetical Operations with Mismatching Index')

cs1 = ContinuousSignal(values, index)
cs2 = ContinuousSignal(values, index + 400)

print('cs1')
print(cs1.values)
print(cs1.index)

print('cs2')
print(cs2.values)
print(cs2.index)

print('\n')

print('cs1 += cs2')
cs1 += cs2
print(cs1.values)

print('\n')

print('cs1 += 1')
cs1 += 1
print(cs1.values)

print('\n')

print('cs1 -= np.ones(index.size)')
cs1 -= np.ones(index.size)
print(cs1.values)

print('\n')

print('cs2 = cs1 + cs1')
cs2 = cs1 + cs1
print(cs2.values)

print('\n')

print('Cubic Spline interpolation fails with Nan(s) values.')
cs1.interpolation_method = 'Cubic Spline'
cs2 = cs1 + cs1
print(cs2.values)

print('\n')

print('Cubic Spline interpolation with filled Nan(s) values.')
cs1.fill_nan()
cs2 = cs1 + cs1
print(cs2.values)


*                                                                             *
*   Empty Object Creation                                                     *
*                                                                             *
cs1[0]
Underlying signal interpolator function does not exists, please ensure you defined both "index" and "values" variables!


cs1[0]
Underlying signal interpolator function does not exists, please ensure you defined both "index" and "values" variables!


cs1[0]
1.0


"index" and "values" variables must have same size!


*                                                                             *
*   Copy Operations                                                           *
*                                                                             *
id(cs1)
139895659685072


id(cs2)
139895659685840


*                                                                             *
*   Item Operations                                            

/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "values" variable is not finite, unpredictable results may occur!
[  6.   8.  10.  12.  14.  16.  nan  nan  nan  nan]
  warn(*args, **kwargs)
/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "values" variable is not finite, unpredictable results may occur!
[  7.   9.  11.  13.  15.  17.  nan  nan  nan  nan]
  warn(*args, **kwargs)
/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "values" variable is not finite, unpredictable results may occur!
[ 12.  16.  20.  24.  28.  nan  nan  nan  nan  nan]
  warn(*args, **kwargs)
/colour-science/colour/colour/utilities/verbose.py:125: UserWarning: "values" variable is not finite, unpredictable results may occur!
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]
  warn(*args, **kwargs)
